## CGM Coordinates

Convert station coordinates from latitude and longitude to altitude adjusted corrected geomagnetic coordinates.

- cgm lat
- cmg lon
- mlt at 0 UT
- declination
- L-shell dipole
- L-shell Geopack IGRF

In [1]:
import igrf12
import aacgmv2
import pandas as pd
import numpy as np
import datetime as datetime

In [2]:
year   = ['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']
alt_km = 100.
stn_f = "C:\\Users\\krmurph1\\Google Drive\\Work\\python\\gmag\\Stations\\station_list.csv"

In [3]:
for yr in year:
    dt = pd.to_datetime(yr)
    
    df = pd.read_csv(stn_f, header=None, skiprows=1, 
                     names = ['array','code','name','latitude','longitude'])

    decl    = np.zeros(df.shape[0])
    cgm_lat = np.zeros(df.shape[0])
    cgm_lon = np.zeros(df.shape[0])
    l_dip   = np.zeros(df.shape[0])
    mlt     = np.zeros(df.shape[0])
    mlt_ut  = np.zeros(df.shape[0])


    # get declination, cgm coords, l-shell
    for index, row in df.iterrows():

        mag = igrf12.igrf(dt,glat=row['latitude'],glon=row['longitude'],alt_km=alt_km)

        cgm_lat[index], cgm_lon[index], cgm_r = aacgmv2.convert_latlon(row['latitude'],row['longitude'], alt_km,dt,code='G2A')

        if cgm_lon[index] <0:
            cgm_lon[index] = 360+cgm_lon[index]

        mlt_ut[index] = aacgmv2.convert_mlt(cgm_lon[index],datetime.datetime(int(yr),1,1,0,0,0),m2a=False)
        mlt[index] = 24- mlt_ut[index]

        l_dip[index] = 1./(np.cos(np.deg2rad(cgm_lat[index]))**2.)

        decl[index] = mag['decl'].values[0]
        if row['code'] == 'GILL':
            print(index,decl[index],cgm_lat[index],cgm_lon[index],l_dip[index])

    df['cgm_latitude']  = cgm_lat
    df['cgm_longitude'] = cgm_lon
    df['declination']   = decl
    df['lshell']        = l_dip
    df['mlt_midnight']  = mlt
    df['mlt_ut']        = mlt_ut

    fn = '.\\Stations\\{0:04d}_station_cgm.txt'.format(int(yr))
    df.to_csv(fn,index=False,float_format="%E", na_rep='NaN')

9 0.3507422544683413 66.28107475891215 332.5431061345665 6.1802700380971745
9 0.24067593422395495 66.22035932901935 332.74059004019415 6.150572511609095
9 0.13181570993443356 66.1593169124843 332.9371238086651 6.1209441377339
9 0.024142638388168335 66.09794913466297 333.1327096889545 6.091387027257041
9 -0.08236184625325349 66.03625760895095 333.3273499967605 6.061903235671791
9 -0.18771586859690775 65.97424393666215 333.5210471127435 6.032494763789942
9 -0.24782958320414214 65.90867671181078 333.7205980233124 6.001648507390674
9 -0.30734148782374593 65.84276658764227 333.91912746746925 5.970894492592324
9 -0.3662604481948755 65.77651532232639 334.11663853759626 5.94023471525009
9 -0.42459515928292196 65.70992465888969 334.31313439124574 5.9096711098131465
9 -0.48235405081963045 65.64299632510509 334.50861824911806 5.879205550178089
9 -0.5300290144922867 65.58419101268854 334.7329293172695 5.852644725275855
9 -0.577323844513926 65.5249612579832 334.95622473995087 5.826086102554234
9 -0

RuntimeError: unable to set time for 2020-01-01 00:00:00

In [ ]:
df.head()

In [ ]:
df[df['code'] == 'GILL']

In [ ]:
pd.set_option('display.max_rows', None) 


In [ ]:
df[(df['lshell']>3) & (df['lshell']<7)][['array','code','cgm_longitude','lshell']]

In [ ]:
df[(df['lshell']>7)][['array','code','cgm_longitude','lshell']]

In [4]:
print(6.893449E+00+1.710655E+01)

23.999999
